In [1]:
import configparser
import boto3
import pandas as pd
import s3fs

# Import your credentials from your .cfg file
config = configparser.ConfigParser()
config.read('aws.cfg')

aws_access_key = config['AWS']['aws_access_key_id']
aws_secret_key = config['AWS']['aws_secret_access_key']

In [2]:
# create s3 client 
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

In [3]:
product = pd.read_csv('s3://techcatalyst-public/resources/sentiment/product.csv')
product.head()

,PRODUCT ID,PRODUCT DESCRIPTION,REVIEW ID
0,1,High-quality smartphone with excellent camera.,R1
1,1,High-quality smartphone with excellent camera.,R2
2,2,Durable laptop with long battery life.,R3
3,2,Durable laptop with long battery life.,R4
4,3,Wireless noise-canceling headphones.,R5


In [4]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   PRODUCT ID           20 non-null     int64 
 1   PRODUCT DESCRIPTION  20 non-null     object
 2   REVIEW ID            20 non-null     object
dtypes: int64(1), object(2)
memory usage: 608.0+ bytes


In [5]:
review = pd.read_csv('s3://techcatalyst-public/resources/sentiment/review.csv')
review.head()

,REVIEW ID,REVIEW TEXT
0,R1,"Great camera quality, but battery life could b..."
1,R2,Very satisfied with this smartphone. Worth eve...
2,R3,Excellent performance and battery life. Highly...
3,R4,Disappointed with the build quality. Feels cheap.
4,R5,Fantastic noise cancellation and sound quality.


In [6]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   REVIEW ID    20 non-null     object
 1   REVIEW TEXT  20 non-null     object
dtypes: object(2)
memory usage: 448.0+ bytes


In [7]:
combined = pd.merge(product, review, how='inner', on='REVIEW ID')
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   PRODUCT ID           20 non-null     int64 
 1   PRODUCT DESCRIPTION  20 non-null     object
 2   REVIEW ID            20 non-null     object
 3   REVIEW TEXT          20 non-null     object
dtypes: int64(1), object(3)
memory usage: 768.0+ bytes


In [10]:
combined.head(50)

,PRODUCT ID,PRODUCT DESCRIPTION,REVIEW ID,REVIEW TEXT
0,1,High-quality smartphone with excellent camera.,R1,"Great camera quality, but battery life could b..."
1,1,High-quality smartphone with excellent camera.,R2,Very satisfied with this smartphone. Worth eve...
2,2,Durable laptop with long battery life.,R3,Excellent performance and battery life. Highly...
3,2,Durable laptop with long battery life.,R4,Disappointed with the build quality. Feels cheap.
4,3,Wireless noise-canceling headphones.,R5,Fantastic noise cancellation and sound quality.
5,3,Wireless noise-canceling headphones.,R6,Not comfortable to wear for long periods.
6,4,Smartwatch with various health tracking features.,R7,Great health tracking features but too expensive.
7,4,Smartwatch with various health tracking features.,R8,Love the design and functionality. Would buy a...
8,5,Portable Bluetooth speaker with rich sound.,R9,Sound quality is amazing for its size.
9,5,Portable Bluetooth speaker with rich sound.,R10,Battery life is too short for outdoor use.


In [46]:
combined['PRODUCT DESCRIPTION'] = combined['PRODUCT DESCRIPTION'].replace('.', '').replace(',', '').astype(str)

In [47]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   PRODUCT ID           20 non-null     int64 
 1   PRODUCT DESCRIPTION  20 non-null     object
 2   REVIEW ID            20 non-null     object
 3   REVIEW TEXT          20 non-null     object
dtypes: int64(1), object(3)
memory usage: 768.0+ bytes


In [33]:
# create comprehend client
comprehend = boto3.client('comprehend',
                          'us-east-1',
                          aws_access_key_id=aws_access_key,
                          aws_secret_access_key=aws_secret_key)

In [38]:
Sentiment = comprehend.detect_sentiment(Text=text, LanguageCode="en")
Sentiment

ClientError: An error occurred (UnrecognizedClientException) when calling the DetectSentiment operation: The security token included in the request is invalid.